In [8]:
import os
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, Dropout, LayerNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.regularizers import L2
from sklearn.model_selection import train_test_split

In [3]:
from PIL import Image

def load_image(path):
    # Load image
    img = Image.open(path)
    
    # Convert to RGBA format if the image has transparency data
    if img.mode == 'P' and 'transparency' in img.info:
        img = img.convert('RGBA')
        
    # Resize image
    img = img.resize((224, 224))
    
    # Convert to numpy array
    img_array = np.array(img)
    
    # Normalize pixel values
    img_array = img_array / 255.0
    
    # Return image array
    return img_array


In [4]:
class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.mha = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = tf.keras.Sequential([
            tf.keras.layers.Dense(ff_dim, activation='relu'),
            tf.keras.layers.Dense(embed_dim)
        ])
        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)

    def call(self, inputs):
        attn_output = self.mha(inputs, inputs)
        attn_output = self.dropout1(attn_output)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output)
        return self.layernorm2(out1 + ffn_output)
        

In [5]:
def create_transformer_model(img_size, num_heads, ff_dim, dropout_rate):
    # Load pre-trained base model
    base_model = tf.keras.applications.MobileNetV2(include_top=False, input_shape=(img_size, img_size, 3))
    x = base_model.output
    
    # Add new convolutional layer with 256 filters
    x = tf.keras.layers.Conv2D(256, kernel_size=3, padding='same', activation='relu')(x)
    
    # Add transformer blocks
    x = TransformerBlock(embed_dim=256, num_heads=num_heads, ff_dim=ff_dim, rate=dropout_rate)(x)
    x = TransformerBlock(embed_dim=256, num_heads=num_heads, ff_dim=ff_dim, rate=dropout_rate)(x)
    
    # Add final layers for binary classification
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dropout(dropout_rate)(x)
    outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)
    
    # Construct model
    model = tf.keras.Model(inputs=base_model.input, outputs=outputs)
    
    # Freeze base model layers
    for layer in base_model.layers:
        layer.trainable = False
    
    return model

In [6]:
import os
from PIL import Image

data_dir = "Detect_solar_dust"

for filename in os.listdir(data_dir):
    if filename.endswith(".png"):
        image_path = os.path.join(data_dir, filename)
        image = Image.open(image_path)
        image = image.convert("RGBA")
        image.save(image_path)


In [7]:
data_dir = 'Detect_solar_dust'

# Get list of all image filenames
filenames = os.listdir(data_dir)

# Shuffle filenames
random.shuffle(filenames)

# Split filenames into training and validation sets
train_filenames, val_filenames = train_test_split(filenames, test_size=0.2, random_state=42)

# Create data generators
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True, validation_split=0.2)
val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255, validation_split=0.2)
train_generator = train_datagen.flow_from_directory(data_dir, target_size=(224, 224), batch_size=32, class_mode='binary', subset='training')
val_generator = val_datagen.flow_from_directory(data_dir, target_size=(224, 224), batch_size=32, class_mode='binary', subset='validation')

# Create transformer model
model = create_transformer_model(img_size=224, num_heads=8, ff_dim=512, dropout_rate=0.2)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


# Train model
history = model.fit(train_generator, epochs=30, validation_data=val_generator)

# Evaluate model on test set
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(data_dir, target_size=(224, 224), batch_size=32, class_mode='binary', shuffle=False)
loss, accuracy = model.evaluate(test_generator)
print('Test loss:', loss)
print('Test accuracy:', accuracy)



Found 2051 images belonging to 2 classes.
Found 511 images belonging to 2 classes.


c:\Users\rishi\anaconda3\envs\.Shit_Sorter\lib\site-packages\PIL\Image.py:979: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch 1/30
 7/65 [==>...........................] - ETA: 1:48 - loss: 2.5824 - accuracy: 0.4955

KeyboardInterrupt: 

In [10]:
# Get list of test filenames and their labels
test_filenames = test_generator.filenames
test_labels = test_generator.labels

# Get predictions for test set
predictions = model.predict(test_generator)

# Print some labelled test images
for i in range(10):
    index = np.random.randint(len(test_filenames))
    filename = test_filenames[index]
    label = test_labels[index]
    prediction = predictions[index][0]
    img = Image.open(os.path.join(data_dir, filename))
    img.show()
    print("Label: ", "Dust" if label else "No dust")
    print("Prediction: ", "Dust" if prediction >= 0.5 else "No dust")
    print()


81/81 [==============================] - 110s 1s/step
Label:  No dust
Prediction:  No dust

Label:  No dust
Prediction:  Dust

Label:  Dust
Prediction:  Dust

Label:  No dust
Prediction:  No dust

Label:  No dust
Prediction:  No dust

Label:  No dust
Prediction:  No dust

Label:  No dust
Prediction:  No dust

Label:  No dust
Prediction:  No dust

Label:  No dust
Prediction:  No dust

Label:  Dust
Prediction:  Dust



In [ ]:
 # out of 10 samples only 1  prediction gone wrong and that too on the image which was a little blur.

In [18]:
def create_cnn_model(img_size, dropout_rate):
    # Load pre-trained base model
    base_model = tf.keras.applications.MobileNetV2(include_top=False, input_shape=(img_size, img_size, 3))
    x = base_model.output
    
    # Add global average pooling layer
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    
    # Add dropout layer
    x = tf.keras.layers.Dropout(dropout_rate)(x)
    
    # Add final layers for binary classification
    outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)
    
    # Construct model
    model = tf.keras.Model(inputs=base_model.input, outputs=outputs)
    
    # Freeze base model layers
    for layer in base_model.layers:
        layer.trainable = False
    
    return model

In [19]:
import os
from PIL import Image

data_dir = "Detect_solar_dust"

for filename in os.listdir(data_dir):
    if filename.endswith(".png"):
        image_path = os.path.join(data_dir, filename)
        image = Image.open(image_path)
        image = image.convert("RGBA")
        image.save(image_path)

# for filename in os.listdir(data_dir):
#     if filename.endswith(".png"):
#         image_path = os.path.join(data_dir, filename)
#         image = image.open(image_path)
#         image = image.convert("RGBA")
# for filename in os.listdir(data_dir):
#     if filename.endswith(".png"):
#         image_path = os.path.join(data_dir, filename)
#         image = image.open(image_path)
#         image = image.convert("RGBA")
#         image.save(image_path)

In [20]:
# Create CNN model
model = create_cnn_model(img_size=224, dropout_rate=0.2)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train model
history = model.fit(train_generator, epochs=30, validation_data=val_generator)

# Evaluate model on test set
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(data_dir, target_size=(224, 224), batch_size=32, class_mode='binary', shuffle=False)
loss, accuracy = model.evaluate(test_generator)
print('Test loss:', loss)
print('Test accuracy:', accuracy)


Epoch 1/30
65/65 [==============================] - 141s 2s/step - loss: 0.6552 - accuracy: 0.6265 - val_loss: 0.5633 - val_accuracy: 0.7182
Epoch 2/30
65/65 [==============================] - 116s 2s/step - loss: 0.5626 - accuracy: 0.7036 - val_loss: 0.5255 - val_accuracy: 0.7260
Epoch 3/30
65/65 [==============================] - 118s 2s/step - loss: 0.5254 - accuracy: 0.7328 - val_loss: 0.4843 - val_accuracy: 0.7554
Epoch 4/30
65/65 [==============================] - 117s 2s/step - loss: 0.5006 - accuracy: 0.7523 - val_loss: 0.4811 - val_accuracy: 0.7632
Epoch 5/30
65/65 [==============================] - 117s 2s/step - loss: 0.4704 - accuracy: 0.7772 - val_loss: 0.4593 - val_accuracy: 0.7828
Epoch 6/30
65/65 [==============================] - 117s 2s/step - loss: 0.4642 - accuracy: 0.7674 - val_loss: 0.5196 - val_accuracy: 0.7632
Epoch 7/30
65/65 [==============================] - 116s 2s/step - loss: 0.4435 - accuracy: 0.7913 - val_loss: 0.4634 - val_accuracy: 0.7789
Epoch 8/30
65